
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


In [199]:
import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np

import pickle

In [200]:
def relabel_axes(old_min, old_max, new_min, new_max):
    scale = (new_max - new_min) / (old_max - old_min)

    def format_func(value, tick_number):
        return new_min + value * scale

    plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))

### grab the data and set up variables

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

LAB_root = "D:\Box\Box\CoganLab" #this is just for jims pc, delete this if using on another computer

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
for sub in subjects:

    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data

    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    # do filtering now
    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    ## remove bad channels
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    good.load_data()

    # CAR
    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


    # make baseline 
    # make stimulus baseline EpochsTFR
    times=[-1, 0.5]
    trials = trial_ieeg(good, "Stimulus", times, preload=True)
    outliers_to_nan(trials, outliers=10) #this removes trial outliers BUT breaks the shape because different stimulus epochs will then have different trial numbers
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    data_dict = {}  # make something to store all the outputs

    for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, event, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
        crop_pad(HG_ev1, "0.5s")

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')
        # HG_ev1.resample(100)
        # HG_ev1.filenames = good.filenames

        # Store the variable in the dictionary with the desired name
        data_dict[f"HG_ev1_{event}"] = HG_ev1
        data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
        
    resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
    resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

    # Plot the evoked data
    fig = resp_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
    fig.savefig(save_dir + '_HG_ev1_Response_rescaled_zscore_no_outliers.png')

    fig = stim_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
    fig.savefig(save_dir + '_HG_ev1_Stimulus_rescaled_zscore_no_outliers.png')

    # decimate for stats
    resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
    stim = data_dict["HG_ev1_Stimulus"]
    HG_base.decimate(2)
    resp.decimate(2)
    stim.decimate(2)

    # do permutation cluster
    resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    
    # plot stats
    plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
    plt.imshow(stim_mask, aspect='auto')  # hold on from matlab
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_stimulus_stats_big.png', dpi=300)

    plt.figure(figsize=(16, 8))
    plt.imshow(resp_mask, aspect='auto')
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_response_stats_big.png', dpi=300)



In [204]:
layout.derivatives

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}

### for testing with a single subject

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
sub = 'D0059'

filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data

save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


# do filtering now
## Crop raw data to minimize processing time
good = crop_empty_data(filt)

## remove bad channels
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


# make baseline 
# make stimulus baseline EpochsTFR
times=[-1, 0.5]

trials = trial_ieeg(good, "Stimulus", times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

data_dict = {}  # Create an empty dictionary

for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_ev1, "0.5s")

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore') #removed .average() part. Check with Aaron if this is okay.
    # HG_ev1.resample(100)
    # HG_ev1.filenames = good.filenames

    # Store the variable in the dictionary with the desired name
    data_dict[f"HG_ev1_{event}"] = HG_ev1
    data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled

resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

# this is broken cuz it's trying to use the .average method from mne that has shape requirements
# resp_evoke_channel_avg = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# resp_evoke_rescaled_channel_avg = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# stim_evoke_channel_avg = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# stim_evoke_rescaled_channel_avg = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))

# # # Manually interpolate NaN values
# resp_evoke.data = np.where(np.isnan(resp_evoke.data), np.interp(resp_evoke.times, resp_evoke.times[~np.isnan(resp_evoke.data[0])], resp_evoke.data[0, ~np.isnan(resp_evoke.data[0])]), resp_evoke.data)

# # Exclude time points with NaN values
# valid_time_points = ~np.isnan(resp_evoke.data[0])  # Filter time points without NaNs
# resp_evoke = resp_evoke.crop(tmin=resp_evoke.times[valid_time_points][0], tmax=resp_evoke.times[valid_time_points][-1])

# Plot the evoked data
fig = resp_evoke_rescaled.plot()
# fig.add_channels([resp_evoke_rescaled_channel_avg])

fig.savefig(save_dir + '_HG_ev1_Response_rescaled_zscore_test.png')

fig = stim_evoke_rescaled.plot()
# fig.add_channels([stim_evoke_rescaled_channel_avg])

fig.savefig(save_dir + '_HG_ev1_Stimulus_rescaled_zscore_test.png')

# decimate for stats
resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
stim = data_dict["HG_ev1_Stimulus"]
HG_base.decimate(2)
resp.decimate(2)
stim.decimate(2)

# import scipy
resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)
stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)

# plot stats
plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
plt.imshow(stim_mask, aspect='auto')
# hold on from matlab
# plt.imshow(x) #plot multiple things like this

relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_stimulus_stats_big.png', dpi=300)

plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
plt.imshow(resp_mask, aspect='auto')
relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_response_stats_big.png', dpi=300)


In [202]:
resp_evoke_rescaled.duration

AttributeError: 'EvokedArray' object has no attribute 'duration'